---
format:
  html:
    embed-resources: true
    code-fold: true
---

## Dimensionality Reduction

## Project Outline

In this section of my project, I hope to utilize some Dimensionality reduction techniques on the data that I gathered from the Riot API to better suit the dataset for future modeling. As seen in the feature selection tab, I was able to find the most relevant variables to use for my Naive Bayes classification modeling. Using what I found from that venture, I hope to compare the feature selection results with results from using Dimensionality Reduction techniques, Principal Component Analysis (PCA) and t-Distributed Stochastic Neighbor Embedding (t-SNE), on the same dataset. After performing the techniques, I want to explore how the variance of the dataset changes through visualizations. This will help me compare the results of my feature selection results with the results from PCA and t-SNE. <br> <br>

![Dimensionality Reduction Image [Source](https://www.sc-best-practices.org/preprocessing_visualization/dimensionality_reduction.html)](images/dimensionality_reduction.png)

__Key Libraries__ <br>
I will be coding these techniques up on Python by using core functions from modules like ```Pandas```, ```Numpy```, and ```Scikit-Learn```. General data management will come from ```Pandas``` and ```Numpy``` while the techniques for ```PCA``` and ```t-SNE``` will mainly come from the ```Scikit-Learn``` library. To create visualizations as mentioned in my outline above, I will use the ```matplotlib``` library to do this. <br> <br>

```python
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
```

__How does this relate to my project?__ <br>
As an aspiring Data Scientist, I hope to go more into Machine Learning, Deep Learning, and AI which means these techniques should be explored somewhat in order to understand fundamental processes that were later advanced. And  with my desire to work on statistical analysis of games, I hope to explore these techniques on my retrieved dataset from the Riot API in order to understand the practicality of these techniques under this topic.

## Dimensionality Reduction Results with PCA

The cleaned data I used: [Retrieved Match Data from Riot API](https://github.com/anly501/dsan-5000-project-shawnhxu/blob/main/data/cleaned_riot_data/league_combined_with_chall_cleaned.csv)

The code for performing PCA: [PCA Code](https://github.com/anly501/dsan-5000-project-shawnhxu/blob/main/codes/05-dim-red/pca.ipynb)


__PCA Process__ <br>
A process of feature extraction where all of the information being used is retained was done on the record data that I retrieved from the Riot API. Firstly, I did some cleaning again to normalize the record data as all features were time-dependent. I also removed the non-numerical variables so that PCA can be done properly. In hindsight, I could have encoded the categorical variables as numerics but I did also want to compare what I found from PCA with what I concluded in Feature Selection so I went with the original time-dependent features. <br><br>

Next, I used ```Scikit-Learn```'s function to fit then transform the dataset to create multiple principal components. These principal components basically contained all information about each feature in some way. The features were combined in some linear fashion to result in a principal component. These transformations were done multiple times to create multiple principal components (created the same number of PC's as the number of features). After creating multiple principal components, they were sorted from highest variance of transformed data points to lowest variance. The first component now having the highest variance, the second component having the second highest variance, and so on. <br><br>

In [ ]:
#define PCA() as instance
#without defining parameters in PCA process first
pca = PCA()

#fit and transform the dataset with PCA 
combined_pca = pca.fit_transform(combined_league_ranks)

#reintroduce the class label (ranks) according to each row.
#the rows should be in the same order as before.
combined_pca = pd.DataFrame(combined_pca)
combined_pca['rank'] = rank_col #will be helpful in visualizing

__Visualizing PCA__ <br>
To visualize this, I created a scatterplot of the first and second principal component to see how varied the points look now after doing PCA transformations on the dataset: <br>

In [ ]:
#assign color to each class label
each_rank = rank_col.unique()
diff_colors = plt.cm.tab10(range(len(each_rank)))
map_color_rank = dict(zip(each_rank, diff_colors))
#now map back to the combined_pca df
combined_pca['rank color'] = combined_pca['rank'].map(map_color_rank)

#plot the PCA transformed dataset into scatterplot
for label in each_rank:
    #subset dataset with each label
    subset = combined_pca[combined_pca['rank']==label]
    plt.scatter(subset[0], subset[1], c=map_color_rank[label], label=label)
#title, labels, and legend
plt.title('Scatterplot of First and Second Principal Component')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.legend()

#save as .png
plt.savefig('all_pc_TRANSFORMED.png')

plt.show()

![1st and 2nd PC Scatterplot](images/first_second_PC.png)

From the plot, the variance of the first principal component's transformed data points is compared to the second principal component. Simply by looking at the x and y-axis, we can see the range of each principal component linearly transformed data points. The first principal component spanning much more and thus would logically have the highest variation. <br><br>

__Visualizing Loadings of First Principal Component__ <br>
After witnessing the variance of the first and second principal component, I wanted to see the loadings of the first principal component. <br><br>

The loadings of each principal component explains how much a feature contributes to the linear transformation of the original dataset. This was done using ```Scikit-Learn```'s PCA module as well since the PCA function they provide also offers a way to see the loadings of each principal component.

In [ ]:
#select the first principal component then get loadings
loadings = pca.components_[0]

#save the loadings into dataframe
load_df = pd.DataFrame({'features': combined_league_ranks.columns, 'loadings': loadings})
#print the loads for each feature
print(load_df)

```
           features  loadings
0         kills/min  0.000035
1        deaths/min -0.000005
2        dmgObj/min  0.132264
3       dmgTurr/min  0.016178
4  vision_score/min -0.000088
5           dmg/min  0.988058
6      dmgTaken/min  0.071493
7       minions/min  0.000296
8          gold/min  0.029582
```

From the output of the loadings for the first principal component, we can see that there are basically no negative loadings meaning almost all features positively contributed to the generation of the first principal component. Additionally, the negative loadings were practically zero. What contributed the most by a mile to the generation of the first principal component was the damage per minute feature from the original dataset. <br><br>

Now to show these values in a visualization: <br>

In [ ]:
#first convert loadings to absolute value as all negatives are basically 0
load_df['absolute loads'] = np.abs(load_df['loadings'])
#next sort by absolute loadings from highest magnitude to lowest
sorted_load = load_df.sort_values(by='absolute loads', ascending=False)

#now plot with features as x-axis and the loadings of each feature
plt.figure(figsize=(10, 6))
plt.bar(sorted_load['features'], sorted_load['loadings'])
plt.title('Loadings of Each Feature for the First Principal Component')
plt.xlabel('Each Feature')
plt.ylabel('Loading Value')
plt.xticks(rotation=45, ha='right')

#save plot as .png
plt.savefig('sorted_loads_PC.png', bbox_inches='tight')

plt.show()


![Loadings Barplot](images/sorted_loads_PC.png)

From the simple barplot, we can confirm that the use of the feature ```"damage per minute"``` definitely contributed the most compared the all other features in the creation of the first principal component. <br> <br>

Now what does this mean? The use of loadings showed us how feature extraction could linearly transform the data points of all features in some way in order to create a well-varied spread of data points while retaining all information within the principal component. To compare to feature selection, this does not tell us which features offer the most information or impact on a Naive Bayes Classification model as that would depend on the transformed data points and their correlation with the class label (or rank in this case). Instead, it offers a method of transforming data to include more variance with reduced dimensionality. This would reduce computation time without losing as much information as feature selection. So in conclusion, we cannot completely compare which features offer the most impact for a classification model aftering doing PCA like what I did with feature selection. That step would have to be taken after transforming the data and training it into a classification model. <br> <br>

__Optimal Number of Principal Components__ <br>
The purpose of finding an optimal number of principal components from PCA is so that the resulting transformed dataset has a usable amount of variance so that a classification model is capable of learning the patterns of the transformed dataset. It is also to lessen the amount of dimensions the dataset has while trying to retain as much information as possible. This trade-off should force us to find the optimal number of principal components that would allow us to keep as much information as possible while reducing the amount of computation needed, lower the risk of overfitting so that models don't learn noise or other specific patterns, and could help us find features that contribute the best to a classification model for example. <br> <br>

Now to do this, I took the cumulative sum of the variance RATIO from the total number of principal components that was captured before. There was an equal number of principal components as features: <br>

In [ ]:
#get the explained_variance_ratio_ and plot the cumulative sum of it
plt.plot(np.arange(1,10), np.cumsum(pca.explained_variance_ratio_))

#x should be simply indexed by the number of principal comps
plt.title('Cumulative Explained Variance RATIO With More Components')
plt.xlabel('# components')
plt.ylabel('cumulative variance ratio')

#save as .png
plt.savefig('optimal_PC_num.png', bbox_inches='tight')

plt.show()

#it seem the elbow first appears at 3 principal components

![Cumulative Sum of PC Variance Ratio](images/optimal_PC_num.png)

Using a simple elbow method of the cumulative sum of the variance ratios of the number of principal components, we can determine what minimal number of principal components should give us the best amount of variance. And from the plot generated, the cumulative sum rate increase starts to diminish at about 3 principal components. Thus, the optimal number of principal components that should be used for PCA should be 3 principal components.


## Dimensionality Reduction Results with t-SNE

The code for performing t-SNE: [t-SNE Code](https://github.com/anly501/dsan-5000-project-shawnhxu/blob/main/codes/05-dim-red/tSNE.ipynb)

__Visualizing t-SNE Distribution__ <br>
Using ```Scikit-Learn```'s library for a t-SNE function, I was able to conveniently transform and fit my record data into a 2-dimensional non-linear version. The idea behind t-SNE is to put more emphasis on data points from the original dataset that are very similar to each other. This is done by measuring two data points similarity through measurements like the Euclidean distance formula. It then compares each data point with all other data points from the original dataset to represent the similarity between points. The model would then adjust the distribution to emphasize similar data points. The t-SNE process then iterates through multiple configurations of points in a low-dimensional space that would best preserve the discovered pairwise similarities of data points found in the higher-dimensional space.
<br> <br>

Another point that should be mentioned with t-SNE is the tuning of the perplexity parameter. With a lower perplexity, the distribution of points follows more heavily with the data points from the original dataset that are close to each other. This is known as following the patterns of the "local" structure. As you go higher in perplexity, the distribution follows more of the "global" structure which means the relationships/patterns of the points that are farther apart in the original dataset are considered more and more. Using a perplexity of somewhere in the middle will help show the relationship of the points while illustrating both the local and global relationships of the original dataset. <br> <br>

To visualize this, I chose to generate a 2-dimensional transformation of the original dataset with t-SNE and plotted the dimensions against each other with varying perplexity parameter values: <br>

In [ ]:
#define perplexity parameter to iterate through
perplex = [5,30,50,100]

#for coloring the plot, convert categorical class labels to numerical representations
labels = rank_col.astype('category').cat.codes

#iterate through perplexity and also generate a plot
count = 0
for per in perplex:

    #count used for subplotting
    count += 1

    #set TSNE with parameters
    tsne = TSNE(n_components=2, perplexity=per, random_state=42) #use same seed for each perplexity case
    #fit/transform with record dataset
    current_tsne = tsne.fit_transform(combined_league_ranks)

    #now we should have a 2 dimensional representation of original dataset
    #plot the TSNE transformation with class labels for coloring
    plt.subplot(2,2,count)
    plt.scatter(current_tsne[:, 0], current_tsne[:, 1], c=labels, cmap='viridis', s=10)
    plt.xlabel('t-SNE First Dimension')
    plt.ylabel('t-SNE Second Dimension')
    plt.title(f't-SNE with Perplexity = {per}')

plt.tight_layout()
#save as .png
plt.savefig('tSNE_perplexities.png', bbox_inches='tight')

plt.show()

![t-SNE with Different Perplexities](images/tSNE_perplexities.png)

From the plot, it definitely seems that a pattern in the relationships of the data points in the higher dimension existed as I increased the perplexity. Immediately at 30 perplexity, there was a curving shape being formed from the originally well spread data points.

## Comparing the Results from PCA and t-SNE

__Effectiveness of Each Technique__ <br>
From the visualizations that I generated for both processes and taking into account the transformations done using both techniques, I feel PCA and t-SNE both had their unique takes on dimensionality reduction. On one hand, PCA had the benefit of being explored in more depth on how each feature can be used to transform the data into different, lower dimensional versions of the original dataset. It was simpler to understand with loadings since the transformation process is linear. On the other hand, t-SNE was able to show the similarity relationships of each data point from the original dataset in a lower dimensional data distribution. t-SNE also had the option to tune the distribution differently with the use of perplexity which allowed for further examination of how farther apart data points contributed in a lower-dimensional space.  <br> <br>

In regards to retaining all information from all features in the original dataset, I think t-SNE showed more potential in preserving the structure of the data better with the use of perplexity as the relationships of all data points can be included with some perplexity. In PCA, the linear combinations sometimes included negative loadings meaning some features actually negatively contributed to the generation of principal components which grows my concern over the use of all features. But I think that is also where the trade-off can occur, since PCA can capture in which direction of the original features offers maximum variance. PCA thus can identify global patterns with this. This is something defintely to consider as there are times when you would want to consider the global structure more or the local structure more. This is an aspect where t-SNE could be better as tuning perplexity down could offer more insight on local structures. Another trade-off that I could think of is that PCA is definitely quicker in computational time since it works with linear operations whereas t-SNE took much longer since it looked into non-linear relationships and similarities. That being said, PCA logically could only look into linear relationships and might not discover non-linear relationships. t-SNE might have a leg up in this case as the process can capture non-linear relationships more.<br><br>

__Visualization with Each Technique__ <br>
For visualizing the generated data from PCA and t-SNE, I would say that PCA might have an upper hand as the visualizations are much easier to understand. The use of linear operations in generating a lower dimensional distribution is much easier to wrap my head around. For example, the well spread distribution of the first and second principal component was pretty easy to understand. The spread of the data points for a principal component simply spread across its associated axis. For t-SNE, the visualization better explained the intricate patterns found in the non-linear generation of the data points. 
<br><br>

__Relating Both Back to Project Topic__ <br>
After going through each technique with the project record dataset, I felt I was not able to uncover more on the global relationship between points with PCA. With the use of t-SNE I was able to see how the relationship of each data point changed with higher perplexity versus the non-unique distribution of the lower perplexity distribution which looked similar to what I found with PCA. With the multiple facets of data being generated in a competitive video game like *League of Legends*, it is understandable that there are more non-linear relationships between features than linear ones. This would thus enable t-SNE to be a much better suited dimensionality reduction technique in my opinion.